# Spark

Hadoop Mapreduce nem elég rugalmas nem teszi lehetővé több map és több reduce egymás utáni futtatását. Magas szintű könyvtár klaszteren kersztüli adatfeldolgozáshoz. Több nyelven is elérhető: scala, python, java. 

In [2]:
from pyspark import SparkConf, SparkContext 

conf = SparkConf() 
sc = SparkContext(conf=conf)

A következő kódrészlettel ellenőrizhető sikerült-e a Spark telepítése.

In [3]:
x = "MWűGkyösdwinko Qau fSJpDaTrykv!w"

rdd = sc.parallelize([x[i:i+2] for i in range(0, len(x)-2, 2)], 2)
res = rdd.map(lambda item: item[0]).collect()

print("".join(res))

Működik a Spark


A spark alapját az RDD nevű adatszerkezet képezi. Ez egy egyszerű interfészen át teszi lehetővé az adatok elosztott feldolgozását.

RDD-t létrehozhatunk például meglévő Python szekvenciákból (pl. lista):

In [4]:
rdd = sc.parallelize([1, 2, 4, 6, 7, 2, 4, 6, 5], 3)

rdd

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:289

Az RDD a klaszter több részén elosztva kerül tárolásra és feldolgozára. Az adatok egy olyan csoportját melyek biztosan azonos gépen vannak partíciónak nevezünk.

In [ ]:
rdd.getNumPartitions() # partíciók számának lekérése

Az RDD adatai újraoszthatók.

In [5]:
rdd2 = rdd.repartition(5)
rdd2.getNumPartitions()

5

Az RDD műveteit két csoportba soroljuk:
- transzformáció: új RDD-t adnak vissza
- akció: valami nem RDD jellegű adatot adnak vissza 

In [6]:
rdd.collect() # adatok összegyűjtése (akció)

[1, 2, 4, 6, 7, 2, 4, 6, 5]

In [7]:
rdd.count() # adatok megszámlálása (akció)

9

In [8]:
rdd.take(5) # első n elem összegyűjtése (akció)

[1, 2, 4, 6, 7]

In [9]:
rdd.filter(lambda number: number % 2 == 0)\
    .reduce(lambda acc, curr: acc + curr)

# filter szűrés (transzformáció)
# reduce hajtogatás (akció)

24

In [10]:
text = sc.textFile('alkotmany.txt') # fájlbeolvasás

In [11]:
text

alkotmany.txt MapPartitionsRDD[13] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
text.take(2)

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.']

Grep:

In [13]:
search_for = 'magyar'
text.filter(lambda line: search_for in line).collect()

['Mi, a magyar nemzet tagjai, az új évezred kezdetén, felelősséggel minden magyarért, kinyilvánítjuk az alábbiakat:',
 'Büszkék vagyunk arra, hogy Szent István királyunk ezer évvel ezelőtt szilárd alapokra helyezte a magyar államot, és hazánkat a keresztény Európa részévé tette.',
 'Büszkék vagyunk a magyar emberek nagyszerű szellemi alkotásaira.',
 'Ígérjük, hogy megőrizzük az elmúlt évszázad viharaiban részekre szakadt nemzetünk szellemi és lelki egységét. Kinyilvánítjuk, hogy a velünk élő nemzetiségek a magyar politikai közösség részei és államalkotó tényezők.',
 'Vállaljuk, hogy örökségünket, egyedülálló nyelvünket, a magyar kultúrát, a magyarországi nemzetiségek nyelvét és kultúráját, a Kárpát-medence természet adta és ember alkotta értékeit ápoljuk és megóvjuk. Felelősséget viselünk utódainkért, ezért anyagi, szellemi és természeti erőforrásaink gondos használatával védelmezzük az utánunk jövő nemzedékek életfeltételeit.',
 'Nem ismerjük el történeti alkotmányunk idegen megszállá

Wordcount:

In [34]:
text.flatMap(lambda line: line.split(" ")).take(2) # flatMap (transzformáció) egy tömböt visszaadó függvényt vár, az össze futtatás tömbjeinek uniójából fog állni a visszaadott RDD

['Mi,', 'a']

A `reduceByKey()` kulcs érték párokból `tuple[K, V]` álló RDD-t vár. Redukciót végez a megadott `V->V` függvény alapján. Egy redukcióban csak azonos kulcsú párok szerepelnek. Egy új RDD-t ad vissza, amiben az összesített kulcs érték párok szerepelnek. Lényegében a MapReduce.

In [39]:
text.flatMap(lambda line: line.split(" "))\
.map(lambda word: (word.replace(''), 1))\
.reduceByKey(lambda count, current: count + current)\
.sortBy(lambda t: t[1], ascending=False)\
.take(10)

[('a', 41),
 ('és', 24),
 ('az', 19),
 ('hogy', 17),
 ('valljuk,', 8),
 ('magyar', 6),
 ('nemzet', 5),
 ('szellemi', 4),
 ('büszkék', 4),
 ('vagyunk', 4)]

A `sortBy()` egy `T` típusú elemeket tartalmazó RDD-t rendez. A megadott `T->S` függvény alapján. Ez a függvény adja meg hogy az összehasonlításnál minden rekord milyen értékkel szerepeljen.